## Problem 1: difference_fanout

>Given a list of numbers, for each number generate a list of the differences between it and $n$ (known as the **fanout** value) following numbers in the list. Return a list of all the lists generated for each number. For members in last part of the list that have less than $n$ following members, calculate as many differences as possible.

``` Python
# example behavior
>>> difference_fanout([3,2,4,6,1], 3)
[[1 ,-1 ,-3], [2, 4, -1], [2, -3], [-5], []]
```
For extra credits (and some extra fun!), try to write your function only using list comprehension. 

### Solution: difference_fanout
``` Python
def difference_fanout(l, fanout):
    """ Return a list of differences for 
        each value with its following terms
        
        Parameters
        ----------
        l: List[Union(int,float)]
            Input list
            
        fanout: int
            Number of neighbors to compute difference with
        
        Returns
        -------
        List[List[int]]
    """
    return [[neighbor - base for neighbor 
             in l[i+1:min(i+1+fanout,len(l))]] 
            for i,base in enumerate(l)]
```

This solution uses two list comprehensions, nested together. For readability, the outer list comprehension should probably be written as a for loop, initializing an empty list and iterating through the input list to append the generated list for each value. 

For each member of the list of index $i$, we compute differences with the slice 
```Python
l[i+1:min(i+1+fanout,len(l))]```
where the `min` function is essential for handling the last few values of the list. Stare at it for awhile until it makes sense! 

### Extension
Recall from [earlier](http://www.pythonlikeyoumeanit.com/Module2_EssentialsOfPython/Functions.html#Functions-are-Objects) that functions are, under the hood, just objects with some special operations that allow you to "call" a function. This means that you can pass other functions as parameters into our function. It is especially powerful since it enables us to generalize the purposes of our functions. For example, we don't have to limit our function to just computing the **difference** between members and their following terms; we can apply **any** *binary operation*. Instead of finding the difference, we can calculate the sum or product or even concatenate two strings for a list of string. The possibility is limitless. 
Armed with this knowledge, we can generalize the code.
```Python
def apply_fanout(l, fanout, op):
    """ Return a list of outputs for each value 
        after applying a binary operation between 
        the value and its following terms
        
        Parameters
        ----------
        l: List[Object]
            Input list
        
        fanout: int
            Number of neighbors to apply the operation with
        
        op: Func(Object, Object) -> Object
            Any binary operation to be applied to members. First
            input should be the member, and the second input should
            be a term following the member.
        
        Returns
        -------
        List[List[Object]]
    """
    return [[op(base, neighbor) for neighbor 
             in l[i+1:min(i+1+fanout,len(l))]] 
            for i,base in enumerate(l)]
```
Now, we can rewrite `difference_fanout` simply as
``` Python
def difference_fanout(l, fanout):
    def compute_inv_diff(a,b):
        return a-b
    return apply_fanout(l, fanout, compute_inv_diff)
```
We can easily change `compute_inv_diff` to some other function for a totally different use. 

## Problem 2: within_margin_percentage
> An algorithm is required to test out what percentage of the parts the factory is producing fall within a safety margin of the design specifications. Given a list of values recording the metrics of the manufactured parts, a list of values representing the desired metrics required by the design, and a margin of error allowed by the design, compute what percentage of the values are within the safety margin (<=)

``` Python
# example behavior
>>> within_margin_percentage([10,5,8,3,2],[9.3,4.2,8.9,3,1.2],0.5)
0.8
```

Only 1 value falls out of the margin of error, as `1.2` is deviating more than `0.5` from `2`. 

### Solution: within_margin_percentage
``` Python
def within_margin_percentage(desired, actual, margin):
    """ Compute the percentage of values that fall within
        a margin of error of the desired values
        
        Parameters
        ----------
        desired: List[Union(int,float)]
            The desired metrics
        
        actual: List[Union(int,float)]
            The actual metrics
        
        margin: Union(int,float)
            The allowed margin of error
        
        Returns
        -------
        float
    """
    count = 0 # counter of how many are within margin
    total = len(desired)
    for i in range(total):
        # if within range, count+=1
        count = count+1 if abs(desired[i]-actual[i]) <= margin else count
    return count / total
```

## Problem 3: is_palindrome
> A palindrome is a string that reads the same from left to right and from right to left. Strings like `racecar` and `Live on time, emit no evil` are palindromes. Notice how only valid alphanumeric characters are accounted for and how palindromes are not case-sensitive. Given a string, return whether or not it is a palindrome. 

``` Python
# example behavior
>>> is_palindrome("Step on no pets!")
True
>>> is_palindrome("'Tis not a palindrome")
False
```

**Tips**: `str.isalnum()` returns whether or not a string has purely alphanumeric characters (it works for one-character strings too)
``` Python
>>> "I love Python".isalnum()
False
>>> "IlovePython".isalnum()
True
```

### Solution: is_palindrome
``` Python
def is_palindrome(s):
    """ Given a string, determine if it is a palindrome.
        Whitespaces and cases are all ignored. Only
        alphanumeric characters are taken into consideration
        
        Parameters
        ----------
        s: Str
            Input string
        
        Returns
        -------
        bool
    """
    filtered_str = "".join([c.lower() if c.isalnum() else "" for c in s])
    
    # the center element in an odd number of characters does not need
    # to be considered
    for i in range(len(filtered_str)//2):
        if filtered_str[i] != filtered_str[-(i+1)]:
            return False
    
    # passing the for loop means all the pairs match up
    return True
```